<a href="https://colab.research.google.com/github/marktfaust/TensorFlow-Neural-Networks/blob/main/Faust_Mark_assignment_7.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Artificial Intelligence
# 464/664
# Assignment #7

## General Directions for this Assignment

00. We're using a Jupyter Notebook environment (tutorial available here: https://jupyter-notebook-beginner-guide.readthedocs.io/en/latest/what_is_jupyter.html),
01. Output format should be exactly as requested (it is your responsibility to make sure notebook looks as expected on Gradescope),
02. Check submission deadline on Gradescope,
03. Rename the file to Last_First_assignment_7,
04. Submit your notebook (as .ipynb, not PDF) using Gradescope, and
05. Do not submit any other files.

## Before You Submit...

1. Re-read the general instructions provided above, and
2. Hit "Kernel"->"Restart & Run All".

## Neural Networks: Architecture

For this assignment we will explore Neural Networks; in particular, we are going to explore model complexity. We will use the same dataset from Assignment #6 to classify a mushroom as either edible ('e') or poisonous ('p'). You are free to use PyTorch, TensorFlow, scikit-learn -- to name a few resources. The goal is to explore different model complexities (architectures) before declaring a winner. Either start with a simple network and make it more complex; or start with a complex model and pare it down. Either way, your submission should clearly demonstrate your exploration.


Your output for each model should look like the output of `cross_validate` from Assignment #6:

```
Fold: 0	Train Error: 15.38%	Validation Error: 0.00%
Fold: 1
...

Mean(Std. Dev.) over all folds:
-------------------------------
Train Error: 100.00%(0.00%) Test Error: 100.00%(0.00%)
```

Notice that "Test Error" has been replaced by "Validation Error." Split your dataset into train, test, and validation sets.


Start with a simple network. Train using the train set. Observe model's performance using the validation set.


Increase the complexity of your network. Train using the train set. Observe model's performance using the validation set.


Model complexity in Assignment #6 was depth limit. You can think of it here as the architecture of the network (number of layers and units per layer). Try at least three different network architectures.


We're trying to find a model complexity that generalizes well. (Recall high bias vs high variance discussion in class.)


Pick the network architecture that you deem best. Use the test set to report your winning model's performance. This is the ONLY time you use the test set.


Try at least three different models; more importantly, document your process: what the results were, how the winning model was determined, what was the winning model's performance on the test data. Clearly highlight these items to receive full credit.

## Importing necessary packages

In [1]:
import pandas as pd
import numpy as np
from statistics import mean
from collections import defaultdict

# Make numpy values easier to read.
np.set_printoptions(precision=3, suppress=True)

import tensorflow as tf
from tensorflow.keras import layers

## Loading and preprocessing agaricus-lepiota.data

In [2]:
# Implementation and exploration.
mushrooms = pd.read_csv("agaricus-lepiota.data", names=["poisonous", "cap-shape", "cap-surface", "cap-color", "bruises", "odor",
           "gill-attachment", "gill-spacing", "gill-size", "gill-color", "stalk-shape", "stalk-root", "stalk-surface-above-ring",
           "stalk-surface-below-ring", "stalk-color-above-ring", "stalk-color-below-ring", "veil-type", "veil-color",
           "ring-number", "ring-type", "spore-print-color", "population", "habitat"])

# Shuffle data
# mushrooms = mushrooms.sample(frac=1).reset_index(drop=True)

mushrooms.head()

,poisonous,cap-shape,cap-surface,cap-color,bruises,odor,gill-attachment,gill-spacing,gill-size,gill-color,...,stalk-surface-below-ring,stalk-color-above-ring,stalk-color-below-ring,veil-type,veil-color,ring-number,ring-type,spore-print-color,population,habitat
0,p,x,s,n,t,p,f,c,n,k,...,s,w,w,p,w,o,p,k,s,u
1,e,x,s,y,t,a,f,c,b,k,...,s,w,w,p,w,o,p,n,n,g
2,e,b,s,w,t,l,f,c,b,n,...,s,w,w,p,w,o,p,n,n,m
3,p,x,y,w,t,p,f,c,n,n,...,s,w,w,p,w,o,p,k,s,u
4,e,x,s,g,f,n,f,w,b,k,...,s,w,w,p,w,o,e,n,a,g


In [3]:
mushroom_features = mushrooms.copy()
mushroom_labels = mushroom_features.pop('poisonous').map({'e': 0, 'p': 1})

In [4]:
inputs = {name: tf.keras.Input(shape=(1,), name=name, dtype=tf.string) for name, column in mushroom_features.items()}

In [5]:
preprocessed_inputs = []

for name, input in inputs.items():
  lookup = layers.StringLookup(vocabulary=np.unique(mushroom_features[name]))
  one_hot = layers.CategoryEncoding(num_tokens=lookup.vocabulary_size())

  x = lookup(input)
  x = one_hot(x)
  preprocessed_inputs.append(x)

In [6]:
preprocessed_inputs_cat = layers.Concatenate()(preprocessed_inputs)

mushroom_preprocessing = tf.keras.Model(inputs, preprocessed_inputs_cat)

# tf.keras.utils.plot_model(model = mushroom_preprocessing, rankdir="LR", dpi=72, show_shapes=True)

In [7]:
mushroom_features_dict = {name: np.array(value)
                          for name, value in mushroom_features.items()}

In [8]:
# features_dict = {name:values[:2] for name, values in mushroom_features_dict.items()}
# print(mushroom_preprocessing(features_dict))
# print()
# print(mushroom_preprocessing(inputs))
# print(inputs)

In [9]:
def evaluate_mushroom_model(body, preprocessing_head, x_train, y_train, inputs, epochs, x_val, y_val):

  preprocessed_inputs = preprocessing_head(inputs)
  result = body(preprocessed_inputs)
  model = tf.keras.Model(inputs, result)

  model.compile(loss=tf.keras.losses.BinaryCrossentropy(from_logits=True),
                optimizer=tf.keras.optimizers.Adam(),
                metrics=["accuracy"])

  return model.fit(x=x_train, y=y_train, epochs=epochs, validation_data=(x_val, y_val))

In [10]:
def create_folds(x_data, y_data, num_folds):
  assert(len(x_data.items()) > 0 and len(y_data) > 0)
  assert(len(list(x_data.values())[0]) == len(y_data))
  k, m = divmod(len(y_data), num_folds)
  print('k', k)
  print('m', m)
  folds = {'x': [], 'y': []}
  for i in range(num_folds):
    start = i * k + min(i, m)
    stop = (i + 1) * k + min(i + 1, m)
    folds['x'].append({name:values[start:stop] for name, values in x_data.items()})
    folds['y'].append(y_data[start:stop])
  return folds['x'], folds['y']

In [11]:
body = tf.keras.Sequential([
    layers.Dense(64, activation='relu'),
    layers.Dense(1)
  ])

num_folds = 12

x_folds, y_folds = create_folds(mushroom_features_dict, mushroom_labels, num_folds)

# --------------------------

for i in range(num_folds):
  print('FOLD NUMBER:', i)
  x_validation, y_validation = x_folds[i], y_folds[i]

  combined_x = defaultdict(lambda: np.array([]))

  for j in range(num_folds):
    if j != i:
      for key, value in x_folds[j].items():
        combined_x[key] = np.concatenate([combined_x[key], value])

  combined_y = [element for fold in y_folds[:i] + y_folds[i+1:] for element in fold]

  x_train = dict(combined_x)
  y_train = pd.Series(combined_y)

  # print(x_train)
  # print(y_train)

  history = evaluate_mushroom_model(body, mushroom_preprocessing, x_train, y_train, inputs, 1, x_validation, y_validation)

# --------------------------

# x_validation = x_folds[0]
# y_validation = y_folds[0]

# print(x_validation)
# print(y_validation)

# history = evaluate_mushroom_model(body, mushroom_preprocessing, x_validation, y_validation, inputs, 2)

# # Get average accuracy
# avg_train_accuracy = mean(history.history['accuracy'])
# # USE EVALUATE
# print('Train Error:', 1 - avg_train_accuracy)
# avg_validation_accuracy = mean(history.history['val_accuracy'])
# print('Validation Error:', 1 - avg_validation_accuracy)

# print(history.history)

# for epoch, val_loss in enumerate(history.history['val_loss']):
#     print(f"Epoch {epoch+1}: validation loss = {val_loss:.4f}")

k 677
m 0
FOLD NUMBER: 0
233/233 ━━━━━━━━━━━━━━━━━━━━ 4s 7ms/step - accuracy: 0.8418 - loss: 0.3052 - val_accuracy: 0.9956 - val_loss: 0.0959
FOLD NUMBER: 1
233/233 ━━━━━━━━━━━━━━━━━━━━ 3s 7ms/step - accuracy: 0.9973 - loss: 0.0170 - val_accuracy: 1.0000 - val_loss: 0.0047
FOLD NUMBER: 2
233/233 ━━━━━━━━━━━━━━━━━━━━ 3s 8ms/step - accuracy: 0.9996 - loss: 0.0030 - val_accuracy: 1.0000 - val_loss: 1.7934e-04
FOLD NUMBER: 3
233/233 ━━━━━━━━━━━━━━━━━━━━ 5s 11ms/step - accuracy: 1.0000 - loss: 5.0806e-04 - val_accuracy: 1.0000 - val_loss: 1.2927e-05
FOLD NUMBER: 4
233/233 ━━━━━━━━━━━━━━━━━━━━ 3s 6ms/step - accuracy: 1.0000 - loss: 1.0530e-04 - val_accuracy: 1.0000 - val_loss: 1.0134e-05
FOLD NUMBER: 5
233/233 ━━━━━━━━━━━━━━━━━━━━ 3s 7ms/step - accuracy: 1.0000 - loss: 2.2235e-05 - val_accuracy: 1.0000 - val_loss: 7.9756e-06
FOLD NUMBER: 6
233/233 ━━━━━━━━━━━━━━━━━━━━ 3s 8ms/step - accuracy: 1.0000 - loss: 7.3041e-06 - val_accuracy: 1.0000 - val_loss: 9.6039e-07
FOLD NUMBER: 7
233/233 ━━━━━━

In [12]:
# mushroom_model.fit(x=mushroom_features_dict, y=mushroom_labels, epochs=10)

## Experiment: Activation Function and Optimizer
Modify the 1) Activation function 2) Optimizer of any chosen model. Try at least one model for each modified component.

Explain the motivation behind the modifications you made.

Explore the effects on the performance.


In [13]:
# Implementation and exploration.

## OPTIONAL. BONUS. Experiment: Loss Function

Modify the loss function of any chosen model.

Explain the motivation behind the modifications you made.

Explore the effects on the performance.


In [14]:
# Implementation and exploration.

No other directions for this assignment, other than what's here and in the "General Directions" section. You have a lot of freedom with this assignment. Don't get carried away. It is expected the results may vary, being better or worse, due to the limitations of the dataset. Graders are not going to run your notebooks. The notebook will be read as a report on how different models were explored. Since you'll be using libraries, the emphasis will be on your ability to communicate your findings.

## Before You Submit...

1. Re-read the general instructions provided above, and
2. Hit "Kernel"->"Restart & Run All".